In [1]:
# Import libraries
import os
import glob
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage
import pandas as pd
import datetime as dt

In [2]:
# Set input directory, and change working directory
inDir = 'D:\Files\School\COMP_490_Senior_Design\Beat-the-Heat\Datasets\LST_BA'
os.chdir(inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(inDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [3]:
LSTFiles = glob.glob('MOD11A1.006_LST_Day_1km**.tif')  # Search for and create a list of LST files
LSTqualityFiles =glob.glob('MOD11A1.006_QC_Day**.tif') # Search the directory for the associated quality .tifs
LSTlut = glob.glob('MOD11A1-006-QC-Day-lookup.csv')    # Search for look up .tifs files
LST_QA_lut = pd.read_csv(LSTlut[0]) # Read in the lut

# Include good quality based on MODLAND
LST_QA_lut = LST_QA_lut[LST_QA_lut['MODLAND'].isin([
    'LST produced, good quality, not necessary to examine more detailed QA', 
    'LST produced, other quality, recommend examination of more detailed QA'])]

# Exclude lower quality VI usefulness
VIU =["LST not produced due to cloud effects, LST not produced primarily due to reasons other than cloud"]
LST_QA_lut = LST_QA_lut[~LST_QA_lut['MODLAND'].isin(VIU)]

LSTgoodQuality = list(LST_QA_lut['Value'])

In [4]:
len(LSTFiles)

731

In [5]:
BAFiles = glob.glob('MCD64A1.006_Burn_Date_**.tif') # Search for and create a list of BA files
BAqualityFiles =glob.glob('MCD64A1.006_QA_**.tif') # Search the directory for the associated quality .tif
BAlut = glob.glob('MCD64A1-006-QA-lookup.csv') # Search for look up table
BA_QA_lut = pd.read_csv(BAlut[0]) # Read in the lut
# Include good quality based on MODLAND
BA_QA_lut = BA_QA_lut[BA_QA_lut['Valid data'].isin([True])]
# Special circumstances unburned
SP =["Too few training observations or insufficient spectral separability between burned and unburned classes"]
BA_QA_lut = BA_QA_lut[~BA_QA_lut['Special circumstances unburned'].isin(SP)]
BA_QA_lut
BAgoodQuality = list(BA_QA_lut['Value']) # Retrieve list of possible QA values from the quality dataframe


In [6]:
len(BAFiles)

24

In [7]:
def getBAFile(BAF, month, year):
    for x in BAF:
        # File name metadata:
        BAproductId = x.split('_')[0] # First: product name
        BAlayerId = x.split(BAproductId + '_')[1].split('_doy')[0] # Second: layer name
        BAyeardoy = x.split(BAlayerId+'_doy')[1].split('_aid')[0] # Third: date
        file_year = dt.datetime.strptime(BAyeardoy, '%Y%j').year
        file_month = dt.datetime.strptime(BAyeardoy, '%Y%j').month
        
        if file_year==year and file_month==month:
            return(x)

In [8]:
LST_result = []
#Process through 731 FM files
for i in range(0, len(LSTFiles)):
    LST = gdal.Open(LSTFiles[i])                               # Read file in, starting with the first LST .tif file
    LSTBand = LST.GetRasterBand(1)                             # Read the band (layer)
    LSTData = LSTBand.ReadAsArray().astype('float')            # Import band as an array with type float
    LSTquality = gdal.Open(LSTqualityFiles[i])                 # Open the first quality file
    LSTqualityData = LSTquality.GetRasterBand(1).ReadAsArray() # Read in as an array
    LSTquality = None
    
    # File name metadata:
    LSTproductId = LSTFiles[i].split('_')[0] # First: product name
    LSTlayerId = LSTFiles[i].split(LSTproductId + '_')[1].split('_doy')[0]     # Second: layer name
    LSTyeardoy = LSTFiles[i].split(LSTlayerId+'_doy')[1].split('_aid')[0]      # Third: date
    LST_aid = LSTFiles[i].split(LSTyeardoy+'_')[1].split('.tif')[0]            # Fourth: unique ROI identif
    LST_date = dt.datetime.strptime(LSTyeardoy, '%Y%j').strftime('%m/%d/%Y')   # Convert YYYYDDD to MM/DD/
    LST_year = dt.datetime.strptime(LSTyeardoy, '%Y%j').year
    LST_month = dt.datetime.strptime(LSTyeardoy, '%Y%j').month
    
    BAFileName = ''
    BAFileName = getBAFile(BAFiles, LST_month, LST_year)
    BAQualityFileName = getBAFile(BAqualityFiles, LST_month, LST_year)

    if not BAFileName in (None, ''):
        BA = gdal.Open(BAFileName)                               # Read file in, starting with MCD64A1 version 6
        BABand = BA.GetRasterBand(1)                             # Read the band (layer)
        BAData = BABand.ReadAsArray().astype('float')            # Import band as an array with type flo
        BAquality = gdal.Open(BAQualityFileName)                 # Open the first quality file
        BAqualityData = BAquality.GetRasterBand(1).ReadAsArray() # Read in as an array
        BAquality = None
        
        # File Metadata
        BA_meta = BA.GetMetadata()                               # Store metadata in dictionary
        LST_meta = LST.GetMetadata()                             # Store metadata in dictionary
        
        # Band metadata
        BAFill = BABand.GetNoDataValue()                        # Returns fill value
        BAStats = BABand.GetStatistics(True, True)              # returns min, max, mean, and standard deviation
        BA = None                                               # Close the GeoTIFF file
        
        # Band metadata
        LSTFill = LSTBand.GetNoDataValue()                      # Returns fill value
        LSTStats = LSTBand.GetStatistics(True, True)            # returns min, max, mean, and standard deviation
        LST = None                                              # Close the GeoTIFF file
        
        BAscaleFactor = float(BA_meta['scale_factor'])          # Search the metadata dictionary for the scale fac
        BAunits = BA_meta['units']                              # Search the metadata dictionary for the units
        BAData[BAData == BAFill] = np.nan                       # Set the fill value equal to NaN for the array
        BAScaled = BAData * BAscaleFactor                       # Apply the scale factor using simple multiplicati
        
        LSTscaleFactor = float(LST_meta['scale_factor'])        # Search the metadata dictionary for the scale factors
        LSTunits = LST_meta['units']                            # Search the metadata dictionary for the units
        LSTData[LSTData == LSTFill] = np.nan                    # Set the fill value equal to NaN for the array
        LSTScaled = LSTData * LSTscaleFactor                    # Apply the scale factor using simple multiplication
        
        # Mask both the LST and BA datasets with good quality values
        BA_masked = np.ma.MaskedArray(BAScaled, np.in1d(BAqualityData, BAgoodQuality, invert = True)) 
        LST_masked = np.ma.MaskedArray(LSTScaled, np.in1d(LSTqualityData, LSTgoodQuality, invert = True))
        BAVal = range(0,367)
        
        # Resampling the BA tif file to match the LST
        BA_resampled = scipy.ndimage.zoom(BA_masked,0.5, order=0)            
        new_col = np.sum(BA_resampled,1).reshape((BA_resampled.shape[0],1))  # Creating a new column to match LST
        BA_resampled = np.append(BA_resampled,new_col,1)                     # Adding new column to BA
        
        # Masking the LST with the BA and extracting LST values
        LST_BA = np.ma.MaskedArray(LST_masked, np.in1d(BA_resampled, BAVal, invert = True)) 
        LST_mean = np.mean(LST_BA)                                   # Obtains the mean of the file
        LST_result.append([LST_date,LST_mean])                       # Add mean to the list
LST_df = pd.DataFrame(LST_result, columns=["Date","LST"])

In [9]:
LST_df.head(50)

,Date,LST
0,01/01/2019,280.170902
1,01/02/2019,281.678667
2,01/03/2019,285.865983
3,01/04/2019,283.249994
4,01/05/2019,275.546052
5,01/06/2019,282.054091
6,01/07/2019,286.090648
7,01/08/2019,283.703392
8,01/09/2019,289.163829
9,01/10/2019,285.854975


In [10]:
LST_df.isnull().count()

Date    731
LST     731
dtype: int64

In [11]:
LST_df.to_csv('LST_BA.csv')